# Rewriting queries with OpenAI Structured Outputs

In [1]:
query = "How do I keep my plants alive while I'm on holiday?"

In [2]:
from pydantic import BaseModel

class RewrittenQuery(BaseModel):
  initial_query: str
  new_query: str

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
from utils import view, SYSTEM_REWRITE

view(SYSTEM_REWRITE)


You are an expert in query expansion and natural language processing. 
Your task is to generate an optimized search query based on the user's input query. 
Follow these guidelines:

1. Analyze the input query for key concepts and intent.
2. Identify any ambiguous terms or phrases that could be clarified.
3. Consider common synonyms, related terms, and alternative phrasings to improve the search.
4. If applicable, expand acronyms or abbreviations.
5. Incorporate any relevant context or domain-specific knowledge.
6. Ensure the expanded query maintains the original intent of the user's question.
7. Prioritize clarity and specificity in the rewritten query.
8. If the original query is already optimal, you may return it unchanged.

Your goal is to produce a single, refined query that will return the best search results. 
The rewritten query should be a natural language question or statement, not a list of keywords.



In [5]:
completion = client.beta.chat.completions.parse(    
  model="gpt-4o-2024-08-06",
  messages=[
    {"role": "system", "content": SYSTEM_REWRITE},
    {"role": "user", "content": query},
  ],
  response_format=RewrittenQuery,
)

In [6]:
view(completion)

ParsedChatCompletion[RewrittenQuery](
    id='chatcmpl-A2lrV6Y5T834K0DCmJdXMRTjSUY0F',
    choices=[
        ParsedChoice[RewrittenQuery](
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ParsedChatCompletionMessage[RewrittenQuery](
                content='{"initial_query":"How do I keep my plants alive while I\'m on holiday?","new_query":"What 
are the best methods for ensuring my indoor and outdoor plants remain healthy and watered during a vacation?"}',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=[],
                parsed=RewrittenQuery(
                    initial_query="How do I keep my plants alive while I'm on holiday?",
                    new_query='What are the best methods for ensuring my indoor and outdoor plants remain healthy 
and watered during a vacation?'
                )
            )
        )
    ],
    created=1725224001,
    mo

In [10]:
def rewrite_query(query, system, response_format):
  completion = client.beta.chat.completions.parse(    
    model="gpt-4o-2024-08-06",
    messages=[
      {"role": "system", "content": SYSTEM_REWRITE},
      {"role": "user", "content": query},
    ],
    response_format=response_format,
  )
  return completion.choices[0].message.parsed.new_query

In [ ]:
## Olympics opening ceremony 

In [11]:
import duckdb

In [12]:
con = duckdb.connect("olympics.duckdb")

In [13]:
con.sql("DESCRIBE olympics")


┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ index       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ embeddings  │ FLOAT[1024] │ YES     │ NULL    │ NULL    │ NULL    │
│ text        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ url         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ title       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [14]:
view(
  con.sql("SELECT index, text FROM olympics LIMIT 10")
)

┌───────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────
────┐
│ index │                                                     text                                                 
│
│ int64 │                                                   varchar                                                
│
├───────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────
────┤
│     0 │ The 2024 Olympics opened in Paris in spectacular style with thousands of athletes sailing along the River
…  │
│     1 │ Swapping a stadium for a waterway for the first time to open the "greatest show on Earth", the near 
four-h…  │
│     2 │ Blue, white and red fireworks had raised the Tricolore above Austerlitz Bridge before 6,800 athletes from
…  │
│     3 │ There were surprise performances through the ceremony, including a cabaret number from US 
singer-songwrite…  │
│     4 │ The day had started with

In [15]:
from search import Search
s = Search(con)

In [16]:
from eval import load_questions

In [17]:
questions = load_questions("data/questions.json")

In [18]:
%%time
new_questions = {
  question: rewrite_query(
    question, SYSTEM_REWRITE, RewrittenQuery
  )
  for question in questions.to_dict()
}

CPU times: user 1.5 s, sys: 79.1 ms, total: 1.58 s
Wall time: 31.3 s


In [19]:
view(new_questions)

{
    'How many competitors are there?': "How many competitors are there in the specific industry or event I'm 
interested in?",
    'How many medals will be won on Saturday?': 'How many medals are expected to be won during the events scheduled
for Saturday at the ongoing sports competition?',
    'How many times has Paris hosted the Olympic games?': 'How many times has the city of Paris, France, been the 
host city for the Olympic Games?',
    'Was Gaga there?': "Was Lady Gaga present at the event or location I'm referring to?",
    'What colors were the fireworks during the ceremony?': 'What were the specific colors of the fireworks 
displayed during the ceremony?',
    'What started the day of the opening ceremony?': 'What event marked the beginning of the day on which the 
opening ceremony took place?',
    'What things went wrong?': 'What problems or issues occurred during the process or event?',
    "What was Serena's role?": "What was Serena Williams' role in the recent tennis t

In [20]:
result1 = {
  question: {
    str(index): score
    for index, score in (s.vector_search(question)
                          .select("index, score")
                          .fetchall()
                        )
  }
  for question in questions.to_dict()
}

In [21]:
result2 = {
  question: {
    str(index): score
    for index, score in (s.vector_search(new_questions[question])
                          .select("index, score")
                          .fetchall()
                        )
  }
  for question in questions.to_dict()
}

In [22]:
from eval import as_run
from ranx import compare

In [23]:
runs = [
  as_run(result1, "Vector"), 
  as_run(result2, "Vector Rewrite")
]

In [24]:
compare(
  questions,
  runs=runs,
  metrics=["hit_rate"],
)


#    Model             Hit Rate
---  --------------  ----------
a    Vector               0.773
b    Vector Rewrite       0.818

In [25]:
from eval import compare_table

In [26]:
view(compare_table(questions, runs))

                             Comparing Retrieval Techniques                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Question                                                   ┃ Vector ┃ Vector Rewrite ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ How many competitors are there?                            │   ✅   │       ✅       │
│ How many medals will be won on Saturday?                   │   ✅   │       ✅       │
│ How many times has Paris hosted the Olympic games?         │   ✅   │       ✅       │
│ Was Gaga there?                                            │   ✅   │       ✅       │
│ What colors were the fireworks during the ceremony?        │   ✅   │       ✅       │
│ What started the day of the opening ceremony?              │   ❌   │       ❌       │
│ What things went wrong?                                    │   ❌   │       ❌       │
│ What was Serena's role?          